# 层级索引

In [1]:
import pandas as pd
import numpy as np

## 多级索引Series

### 笨方法

In [2]:
index = [('California', 2000), ('California', 2010), ('New York', 2000),
         ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2000):('New York', 2010)]

# 通过元组构成的多级索引
# 直接在 Series 上取值或用切片查询数据

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2000]]

# 选出2000年的数据变得很麻烦

(California, 2000)    33871648
(New York, 2000)      18976457
(Texas, 2000)         20851820
dtype: int64

### Pandas多级索引

In [7]:
# 创建多层索引对象
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [9]:
index.levels

# levels 属性表示索引的等级

FrozenList([['California', 'New York', 'Texas'], [2000, 2010]])

In [10]:
# 基于索引，重排对象
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [11]:
# 多级索引访问
pop[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

### 高维数据的多级索引

In [12]:
# unstack() 方法将多级索引 Series 转换成 DataFrame

pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [13]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [14]:
# 利用 multi-indexing ,
# 在 Series 里呈现两维数据；同理，在 Series 或 DataFrame 里呈现更高维数据

pop_df = pd.DataFrame({
    'under18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014],
    'total':
    pop
})
pop_df

under18     total
California 2000  9267089  33871648
           2010  9284094  37253956
New York   2000  4687374  18976457
           2010  4318033  19378102
Texas      2000  5906301  20851820
           2010  6879014  25145561

In [15]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## 多级索引的创建方法

### 显式地创建多级索引

In [16]:
#  将 index 参数设置为至少二维的索引数组

df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.452637  0.359713
  2  0.162516  0.571430
b 1  0.155037  0.346884
  2  0.092077  0.810378

In [17]:
#  将元组作为键的字典传递给

data = {
    ('California', 2000): 33871648,
    ('California', 2010): 37253956,
    ('Texas', 2000): 20851820,
    ('Texas', 2010): 25145561,
    ('New York', 2000): 18976457,
    ('New York', 2010): 19378102
}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

In [18]:
# 显式地创建 MultiIndex 也是很有用的

pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [19]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [20]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### 多级索引的等级名称

In [21]:
# MultiIndex 构造器中通过 names 参数设置等级名，
# 也可以在创建之后通过索引的 names 属性来修改名称

pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 多级列索引
每个 DataFrame 的行与列都是对称的，
也就是 既然有多级行索引，那么同样可以有 **多级列索引**

In [22]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      52.0  35.2  47.0  36.9  31.0  36.3
     2      32.0  39.5  39.0  36.6  31.0  36.0
2014 1      39.0  37.9  40.0  39.0  39.0  39.2
     2      65.0  37.1  40.0  38.2  35.0  35.5

In [23]:
health_data['Bob']

# 如果想获取包含多种标签的数据，需要通过对多个维度的多次查询才能实现，
# 这时使用多级行列索引进行查询会非常方便

type          HR  Temp
year visit            
2013 1      30.0  37.1
     2      46.0  36.8
2014 1      44.0  37.0
     2      19.0  39.2

## 多级索引的取值与切片 

### Series多级索引

In [23]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [24]:
# 可以通过对多个级别索引值获取单个元素

pop['California', 2000]

33871648

In [28]:
# 也支持局部取值（partial indexing），即只取索引的某一个层级
# 未被选中的低层索引值会被保留

pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [29]:
# 局部切片

pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [30]:
# 用较低层级的索引取值，第一层级的索引可以用空切片

pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [31]:
# 通过布尔掩码选择数据

pop[pop > 22000]

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [32]:
# fancy indexing 

pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### DataFrame多级索引

In [33]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      52.0  35.2  47.0  36.9  31.0  36.3
     2      32.0  39.5  39.0  36.6  31.0  36.0
2014 1      39.0  37.9  40.0  39.0  39.0  39.2
     2      65.0  37.1  40.0  38.2  35.0  35.5

In [34]:
# 由于 DataFrame 的基本索引是列索引，因此 Series 中多级索引
# 的用法到了 DataFrame 中就应用在列上

health_data['Guido', 'HR']

year  visit
2013  1        47.0
      2        39.0
2014  1        40.0
      2        40.0
Name: (Guido, HR), dtype: float64

In [35]:
# iloc 访问，本质将多维 df 当二维数组处理

health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      52.0  35.2
     2      32.0  39.5

In [36]:
# 使用索引元组，但是元组内部中切片会导致语法错误
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        52.0
      2        32.0
2014  1        39.0
      2        65.0
Name: (Bob, HR), dtype: float64

In [37]:
# 使用索引元组，但是元组内部中切片会导致语法错误

health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-37-d683bc595694>, line 3)

In [50]:
# 创建索引切片对象，实现在行和列分别进行切片

idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,35.0,33.0,41.0
2014,1,48.0,35.0,53.0


## 多级索引行列转换

### 有序和无序的索引 
> **如果 MultiIndex 不是有序的索引，那么大多数切片操作都会失败**

In [38]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data

a  1    0.728333
   2    0.756556
c  1    0.790172
   2    0.300610
b  1    0.993282
   2    0.680417
dtype: float64

In [39]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)
    
# MultiIndex 无序排列    

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [40]:
# 排序
data = data.sort_index()
data

a  1    0.728333
   2    0.756556
b  1    0.993282
   2    0.680417
c  1    0.790172
   2    0.300610
dtype: float64

In [41]:
# 排序后切片成功

data['a':'b']

a  1    0.728333
   2    0.756556
b  1    0.993282
   2    0.680417
dtype: float64

### 索引stack与unstack
通过 level 参数设置转换的索引层级

In [42]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [43]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [44]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 索引的设置与重置
层级数据维度转换的另一种方法是行列标签转换，可以通过reset_index 方法实现

In [45]:
pop_flat = pop.reset_index(name="population")
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


将上述的原始输入数据的列直接转换成 MultiIndex,可以通过DataFrame 的 set_index 方法实现

In [47]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## 多级索引的数据累计方法
- 一些 Pandas 自带的数据累计方法，比如mean()、sum() 和 max()。
- 而对于层级索引数据，可以设置参数level 实现对数据子集的累计操作。

In [49]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      52.0  35.2  47.0  36.9  31.0  36.3
     2      32.0  39.5  39.0  36.6  31.0  36.0
2014 1      39.0  37.9  40.0  39.0  39.0  39.2
     2      65.0  37.1  40.0  38.2  35.0  35.5

In [50]:
# 按 行 进行操作
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     42.0  37.35  43.0  36.75  31.0  36.15
2014     52.0  37.50  40.0  38.60  37.0  37.35

In [51]:
# 指定 axis 参数,按 列 进行操作
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,38.666667,36.750000
2014,43.000000,37.816667
